In [ ]:
#| default_exp clinical_tutor

# Clinical Tutor

> Core module for using learning context for context-appropriate tutor responses


## Setup

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Dict, List, Optional, Any, Tuple
import os
import json
import logging
from pathlib import Path
from datetime import datetime
from dotenv import load_dotenv
import aiohttp
import re
from collections import defaultdict
from wardbuddy.learning_context import LearningContext, setup_logger

# Load environment variables
load_dotenv()

logger = setup_logger(__name__)

C:\Users\deepa\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Adaptive Clinical Tutor

This module implements:

 - Engages in natural case discussions like a clinical supervisor
 - Provides context-aware feedback based on student's rotation and preferences
 - Analyzes discussions to track learning progress
 - Integrates with the student's learning context

The tutor aims to mimic real-world clinical teaching interactions where students present cases and receive feedback in a natural conversational style.


In [ ]:
#| export 
class OpenRouterException(Exception):
    """Custom exception for OpenRouter API errors"""
    pass

In [ ]:
#| export
class ClinicalTutor:
    """
    Adaptive clinical teaching module that provides context-aware feedback.
    
    The tutor acts as an experienced clinical supervisor, engaging in natural
    case discussions while tracking student progress and adapting feedback
    based on learning context.
    
    Attributes:
        learning_context (LearningContext): Student's learning context
        model (str): LLM model identifier
        api_url (str): OpenRouter API endpoint
    """
    
    def __init__(
        self,
        context_path: Optional[Path] = None,
        model: str = "anthropic/claude-3.5-sonnet"
    ):
        """
        Initialize clinical tutor.
        
        Args:
            context_path: Optional path for context persistence
            model: Model identifier for OpenRouter
        """
        self.api_key: str = os.getenv("OPENROUTER_API_KEY")
        if not self.api_key:
            raise ValueError("OpenRouter API key not found")
        
        self.api_url: str = "https://openrouter.ai/api/v1/chat/completions"
        self.model: str = model
        
        self.learning_context = LearningContext(context_path)
        self.context_path = context_path
        
        # Track conversation state
        self.current_case: Dict = {
            "started": None,
            "chief_complaint": None,
            "key_findings": [],
            "assessment": None,
            "plan": None
        }
        
        logger.info(f"Clinical tutor initialized with model: {model}")
    
    async def _get_completion(
        self,
        messages: List[Dict],
        temperature: float = 0.7,
        max_retries: int = 3
    ) -> str:
        """
        Get completion from OpenRouter API with retry logic.
        
        Args:
            messages: List of conversation messages
            temperature: Temperature for response generation
            max_retries: Maximum retry attempts
            
        Returns:
            str: Model response
            
        Raises:
            OpenRouterException: If API calls fail after retries
        """
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json",
            "HTTP-Referer": "http://localhost:7860"
        }
        
        data = {
            "model": self.model,
            "messages": messages,
            "temperature": temperature,
            "max_tokens": 2000
        }
        
        for attempt in range(max_retries):
            try:
                async with aiohttp.ClientSession() as session:
                    async with session.post(
                        self.api_url,
                        headers=headers,
                        json=data,
                        timeout=30
                    ) as response:
                        response.raise_for_status()
                        result = await response.json()
                        return result["choices"][0]["message"]["content"]
                        
            except Exception as e:
                if attempt == max_retries - 1:
                    raise OpenRouterException(f"API call failed: {str(e)}")
                logger.warning(f"Retry {attempt + 1} after error: {str(e)}")
                # Could add exponential backoff here if needed
    
    def _build_discussion_prompt(self) -> str:
        """Build context-aware prompt for case discussion."""
        rotation = self.learning_context.current_rotation
        active_preferences = [
            p["focus"] for p in self.learning_context.feedback_preferences 
            if p["active"]
        ]
        
        significant_gaps = {
            topic: score for topic, score 
            in self.learning_context.knowledge_profile["gaps"].items()
            if score < 0.7  # Only include significant gaps
        }
        
        prompt = f"""You are an experienced clinical supervisor in {rotation['specialty']}. Act as an engaging and conversational tutor who coaches towards deeper understanding through Socratic dialogue and targeted questions.

        Key Principles:
        1. Assume I have strong foundational knowledge in medicine, clinical reasoning, and pre-medical sciences
        2. Focus on high-level connections and nuanced clinical decision-making
        3. Use targeted questions to explore my thought process and highlight key learning points
        4. Share relevant clinical pearls and real-world applications
        5. Be conversational and engaging, avoiding lecture-style responses
        
        Current Rotation Focus Areas:
        {', '.join(rotation['key_focus_areas'])}

        Areas for Deep Dive:
        {', '.join(f'{topic} (confidence: {score:.1f})' for topic, score in significant_gaps.items()) if significant_gaps else 'General clinical reasoning'}

        Student's Interests:
        {', '.join(active_preferences) if active_preferences else 'Broad clinical discussion'}

        Ask probing questions that explore clinical reasoning and highlight important connections. I will ask for clarification 
        if concepts need more explanation."""

        return prompt
    
    def _build_analysis_prompt(self, conversation: List[Dict[str, str]]) -> str:
        """
        Build prompt for post-discussion analysis.
        
        Args:
            conversation: List of message dictionaries with roles and content
            
        Returns:
            str: Analysis prompt
        """
        # Extract case details
        case_content = ""
        for msg in conversation:
            if msg["role"] == "user":
                case_content += msg["content"] + "\n"
        
        return f"""Analyze the following case discussion between a medical student and 
        clinical supervisor. Focus on the student's demonstrated knowledge, skills, 
        and areas for improvement.

        Case Content:
        {case_content}

        Please identify:
        1. Key clinical concepts and learning points demonstrated or discussed
        2. Areas where the student showed uncertainty or knowledge gaps
        3. Strengths demonstrated in clinical reasoning and presentation
        4. Specific learning objectives that would help the student's development

        Frame your response to help with ongoing learning:
        - Start with positive observations
        - Be specific about knowledge gaps
        - Make concrete suggestions for improvement
        - Connect to practical clinical scenarios"""
    
    async def discuss_case(
        self, 
        message: str,
        temperature: float = 0.7
    ) -> str:
        """
        Natural case discussion with context-aware responses.
        
        Args:
            message: Student's input message
            temperature: Temperature for response generation
            
        Returns:
            str: Clinical supervisor's response
        """
        try:
            # Update case tracking
            if not self.current_case["started"]:
                self.current_case["started"] = datetime.now()
                # Try to identify chief complaint from first message
                cc_match = re.search(r"(\d+)\s*[yY][oO]\s*[MmFf]\s*with\s*([^.]*)", message)
                if cc_match:
                    self.current_case["chief_complaint"] = cc_match.group(2).strip()
            
            # Build system prompt
            system_prompt = self._build_discussion_prompt()
            
            messages = [{
                "role": "system",
                "content": system_prompt
            }, {
                "role": "user",
                "content": message
            }]
            
            response = await self._get_completion(messages, temperature)
            return response
            
        except Exception as e:
            logger.error(f"Error in case discussion: {str(e)}")
            return "I apologize, but I encountered an error. Please try presenting your case again."
    
    async def analyze_discussion(
        self,
        conversation: List[Dict[str, str]]
    ) -> Dict[str, Any]:
        """
        Analyze completed case discussion for learning insights.
        
        Args:
            conversation: List of message dictionaries with roles and content
            
        Returns:
            dict: Analysis results containing:
                - learning_points: List of key concepts learned
                - gaps: Dict of identified knowledge gaps
                - strengths: List of demonstrated strengths
                - suggested_objectives: List of recommended learning goals
        """
        try:
            # Reset case tracking
            self.current_case = {
                "started": None,
                "chief_complaint": None,
                "key_findings": [],
                "assessment": None,
                "plan": None
            }
            
            # Get analysis from model
            analysis_prompt = self._build_analysis_prompt(conversation)
            messages = [{
                "role": "system",
                "content": analysis_prompt
            }]
            messages.extend(conversation)
            
            response = await self._get_completion(messages, temperature=0.3)
            
            # Parse insights
            insights = self._parse_analysis(response)
            
            # Update learning context
            self._update_context_from_analysis(insights)
            
            return insights
            
        except Exception as e:
            logger.error(f"Error in discussion analysis: {str(e)}")
            return {
                "learning_points": [],
                "gaps": {},
                "strengths": [],
                "suggested_objectives": []
            }
    
    def _parse_analysis(self, response: str) -> Dict[str, Any]:
        """
        Parse analysis response into structured insights.
        
        Uses pattern matching and basic NLP to extract:
        - Learning points (key concepts discussed)
        - Knowledge gaps with confidence estimates
        - Demonstrated strengths
        - Suggested learning objectives
        
        Args:
            response: Raw analysis response
            
        Returns:
            dict: Structured analysis insights
        """
        insights = {
            "learning_points": [],
            "gaps": {},
            "strengths": [],
            "suggested_objectives": []
        }
        
        try:
            # Split into sections
            sections = response.lower().split("\n\n")
            
            for section in sections:
                if "learning point" in section or "key concept" in section:
                    # Extract bullet points or numbered items
                    points = re.findall(r"[-•*]\s*(.+)$", section, re.MULTILINE)
                    insights["learning_points"].extend(points)
                    
                elif "gap" in section or "uncertainty" in section:
                    # Look for topic mentions with confidence indicators
                    gaps = re.findall(
                        r"(limited|uncertain|unclear|difficulty with)\s+([^,.]+)", 
                        section
                    )
                    for indicator, topic in gaps:
                        # Estimate confidence based on language
                        confidence = 0.4 if "limited" in indicator else 0.6
                        insights["gaps"][topic.strip()] = confidence
                        
                elif "strength" in section or "demonstrated" in section:
                    # Extract positive mentions
                    strengths = re.findall(r"[-•*]\s*(.+)$", section, re.MULTILINE)
                    insights["strengths"].extend(strengths)
                    
                elif "objective" in section or "suggest" in section:
                    # Extract recommended objectives
                    objectives = re.findall(r"[-•*]\s*(.+)$", section, re.MULTILINE)
                    insights["suggested_objectives"].extend(objectives)
            
            return insights
            
        except Exception as e:
            logger.error(f"Error parsing analysis: {str(e)}")
            return insights
    
    def _update_context_from_analysis(self, insights: Dict[str, Any]) -> None:
        """
        Update learning context based on discussion analysis.
        
        Args:
            insights: Dictionary of analysis insights
        """
        try:
            # Update knowledge gaps
            for topic, confidence in insights["gaps"].items():
                self.learning_context.update_knowledge_gap(topic, confidence)
            
            # Add strengths
            for strength in insights["strengths"]:
                self.learning_context.add_strength(strength)
            
            # Save context if path provided
            if self.context_path:
                self.learning_context.save_context(self.context_path)
                
        except Exception as e:
            logger.error(f"Error updating context: {str(e)}")

## Tests

In [ ]:
async def test_clinical_tutor():
    """Test ClinicalTutor functionality"""
    if not os.getenv("OPENROUTER_API_KEY"):
        print("Skipping tests: No API key")
        return
        
    tutor = ClinicalTutor()
    
    # Test case discussion
    test_case = """
    28yo M with chest pain
    - 2 days duration
    - Sharp, pleuritic
    - No fever or cough
    - Vitals stable
    - Clear exam
    A/P: Likely MSK pain
    """
    
    try:
        # Test basic discussion
        response = await tutor.discuss_case(test_case)
        assert isinstance(response, str)
        assert len(response) > 0
        
        # Only assert case tracking if chief complaint was detected
        if tutor.current_case["chief_complaint"]:
            assert "chest pain" in tutor.current_case["chief_complaint"].lower()
        
        print("Discussion test passed")
        
        # Test discussion analysis
        conversation = [
            {"role": "user", "content": test_case},
            {"role": "assistant", "content": response}
        ]
        
        analysis = await tutor.analyze_discussion(conversation)
        assert isinstance(analysis, dict)
        assert all(k in analysis for k in [
            'learning_points', 'gaps', 'strengths', 'suggested_objectives'
        ])
        print("Analysis test passed")
        
    except Exception as e:
        print(f"Test failed: {str(e)}")
        raise
    
    print("All clinical tutor tests passed!")

# Run tests
if __name__ == "__main__":
    import asyncio
    if not asyncio.get_event_loop().is_running():
        asyncio.run(test_clinical_tutor())

RuntimeError: asyncio.run() cannot be called from a running event loop